### First, let's parse the MCAT/GPA data (from shemassian's website)

Source: https://www.shemmassianconsulting.com/blog/average-gpa-and-mcat-score-for-every-medical-school

Note: I manually updated Vanderbilt's average MCAT/GPA since it wasn't correct on the website.
Note: I manually updated Michigan's average MCAT/GPA since it wasn't correct on the website.


In [1]:
import pandas as pd
import os

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"

# Point to the NEW .html file you just saved
html_path = os.path.join(ROOT_DIR, "data/raw_inputs/Average GPA and MCAT Score for Every Medical School (2025) — Shemmassian Academic Consulting.html") 

# This code should work perfectly now
dfs = pd.read_html(html_path, attrs={'id': 'myTable'})
df_mcat_gpa = dfs[0]
df_mcat_gpa.drop(columns=['Minimum MCAT'], inplace=True)
df_mcat_gpa.sort_values(by='Average MCAT', inplace=True, ascending=False)
df_mcat_gpa # Drop any rows where Average MCAT is not a number

df_mcat_gpa[df_mcat_gpa['Medical School']=='Vanderbilt University School of Medicine']
df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='Vanderbilt University School of Medicine', 'Average MCAT'] = 522
df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='Vanderbilt University School of Medicine', 'Average GPA'] = 3.94

df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='University of Michigan Medical School*', 'Average MCAT'] = 517
df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='University of Michigan Medical School*', 'Average GPA'] = 3.92

df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='Louisiana State University – New Orleans School of Medicine*', 'Average MCAT'] = 509
df_mcat_gpa.loc[df_mcat_gpa['Medical School']=='Louisiana State University – New Orleans School of Medicine*', 'Average GPA'] = 3.76

df_mcat_gpa = df_mcat_gpa[pd.to_numeric(df_mcat_gpa['Average MCAT'], errors='coerce').notnull()]
# Same for 'Average GPA'.
# Then we want two-level sorting: first by Average MCAT (descending), then by Average GPA (descending)
df_mcat_gpa = df_mcat_gpa[pd.to_numeric(df_mcat_gpa['Average GPA'], errors='coerce').notnull()]
df_mcat_gpa['Average MCAT'] = pd.to_numeric(df_mcat_gpa['Average MCAT'])
df_mcat_gpa['Average GPA'] = pd.to_numeric(df_mcat_gpa['Average GPA'])
df_mcat_gpa.sort_values(by=['Average MCAT', 'Average GPA'], inplace=True, ascending=[False, False])
df_mcat_gpa.reset_index(drop=True, inplace=True)
# Convert 'Medical School' to lower case for easier merging later
df_mcat_gpa['Medical School'] = df_mcat_gpa['Medical School'].str.lower()
df_mcat_gpa['Medical School'] = df_mcat_gpa['Medical School'].str.replace('*', '', regex=False).str.strip()

### Next, lets parse Debt Information from MSAR

Source: https://students-residents.aamc.org/medical-school-admission-requirements/medical-school-admission-requirements-reports-applicants-and-advisors

In [2]:
import camelot
import pandas as pd
import os

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"

file_path = os.path.join(ROOT_DIR, 'data/raw_inputs/MSAR014 - MSAR Debt  Information.pdf')

# 1. Change pages to the full range
tables = camelot.read_pdf(file_path, pages='2-7', flavor='lattice')

processed_dfs = [] # 2. Create a list to hold all our DFs

if tables.n > 0:
    print(f"Found {tables.n} tables across pages 2-7.")
    
    # 3. Loop through every table object
    for table in tables:
        my_df = table.df
        
        # Apply your exact header logic to each table
        try:
            my_df.columns = my_df.iloc[0]
            my_df = my_df[1:]
            processed_dfs.append(my_df)
        except Exception as e:
            print(f"Warning: Could not process a table, possibly bad format? Error: {e}")
            print(f"Problematic table data:\n{table.df.head()}\n")
    
    # 4. Stack all the individual DataFrames into one
    if processed_dfs:
        df_debt_info = pd.concat(processed_dfs, ignore_index=True)
    else:
        df_debt_info = pd.DataFrame() # Empty
        
else:
    print("No tables found in that page range.")
    df_debt_info = pd.DataFrame()

df_debt_info['Average Graduate\nIndebtedness'] = pd.to_numeric(df_debt_info['Average Graduate\nIndebtedness'].str.replace('[\$,]', '', regex=True), errors='coerce')
df_debt_info = df_debt_info[df_debt_info['Average Graduate\nIndebtedness'].notnull()]
df_debt_info.sort_values(by='Average Graduate\nIndebtedness', ascending=True, inplace=True)
df_debt_info.reset_index(drop=True, inplace=True)
df_debt_info.head(30)

# Let's replace any '*' characters in the 'Medical School' column with an empty string
df_debt_info['Medical School'] = df_debt_info['Medical School'].str.replace('*', '', regex=False).str.strip()
# Now turn the 'Medical School' to lower case for easier matching later
df_debt_info['Medical School'] = df_debt_info['Medical School'].str.lower()


<>:40: SyntaxWarning: invalid escape sequence '\$'
<>:40: SyntaxWarning: invalid escape sequence '\$'
/var/folders/76/y_3kwb_54mx0bh961wd_ycb00000gn/T/ipykernel_52969/2777296739.py:40: SyntaxWarning: invalid escape sequence '\$'
  df_debt_info['Average Graduate\nIndebtedness'] = pd.to_numeric(df_debt_info['Average Graduate\nIndebtedness'].str.replace('[\$,]', '', regex=True), errors='coerce')


Found 6 tables across pages 2-7.


### Next, let's parse NIH funding data from Blue Ridge Institute for Medical Research

Source: https://brimr.org/

In [3]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/SchoolOfMedicine_2024_C.xlsx")
df_nih_funding = pd.read_excel(file_path, sheet_name='2024', header=1)
# Currently the 'Name' column is in caps lock, let's make it lower case
df_nih_funding['Name'] = df_nih_funding['Name'].str.lower()
df_nih_funding.head(25)

# However, we're gonna have to add info for some schools manually. Harvard comes to mind (mass general). 

manual_entries_2024 = [
    {'Name': 'massachusetts general hospital', 'School of Medicine Award': 655235087},
    {'Name': 'brigham and womens hospital', 'School of Medicine Award': 388162121},
    {'Name': 'boston childrens hospital', 'School of Medicine Award': 229894668},
    {'Name': 'beth israel deaconess medical center', 'School of Medicine Award': 119860535}
]
df_manual_2024 = pd.DataFrame(manual_entries_2024)
df_nih_funding = pd.concat([df_nih_funding, df_manual_2024], ignore_index=True)
df_nih_funding.reset_index(drop=True, inplace=True)

### Next, lets parse the Faculty Data from AAMC
Source: https://www.aamc.org/data-reports/faculty-institutions/report/faculty-roster-us-medical-school-faculty

In [4]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/U.S. Medical School Faculty, 2024.xlsx")
df_faculty_info = pd.read_excel(file_path, sheet_name='USMSF Table 2', header=3)
df_faculty_info = df_faculty_info.iloc[:-3]
df_faculty_info.rename(columns={'Total': 'Total Faculty'}, inplace=True)

### Next, let's parse the USNWR Hospital Rankings Data

Got using the usnwr_scraper.ipynb

In [5]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_hospital_rankings.csv")
df_hospital_rankings_raw = pd.read_csv(file_path)
file_path_pivoted_path = os.path.join(ROOT_DIR, "data/hospital_rankings_pivoted.csv")
df_hospital_rankings_pivoted = pd.read_csv(file_path_pivoted_path)
df_hospital_rankings_pivoted # Let's set the 'Institution' to be all lower case
df_hospital_rankings_pivoted['Institution'] = df_hospital_rankings_pivoted['Institution'].str.lower()

### Next, Let's Parse Diversity Data

**Source: https://www.aamc.org/data-reports/data/2024-facts**, Table B-5.1

In [6]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/2024_FACTS_Table_B-5.1 - Updated.xlsx")
df_diversity_info = pd.read_excel(file_path, sheet_name='FACTS Table B-5.1', header=4)
df_diversity_info = df_diversity_info[df_diversity_info['Total \nEnrollment'].notnull()]
df_diversity_info.rename(columns={'Total Enrollment': 'State', 'Unnamed: 1': 'AAMC_Institution'}, inplace=True)
df_diversity_info.rename(columns={'Total \nEnrollment': 'Total Enrollment'}, inplace=True)
# We want to make a column called URM%, which is the summed of 'American Indian \nor Alaska Native', 'Black or \nAfrican American', 'Hispanic, Latino, or \nof Spanish Origin', 'Native Hawaiian or \nOther Pacific Islander', and then divided by 'Total Enrollment'
df_diversity_info['American Indian \nor Alaska Native'] = pd.to_numeric(df_diversity_info['American Indian \nor Alaska Native'], errors='coerce').fillna(0)
df_diversity_info['Black or \nAfrican American'] = pd.to_numeric(df_diversity_info['Black or \nAfrican American'], errors='coerce').fillna(0)
df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '] = pd.to_numeric(df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '], errors='coerce').fillna(0)
df_diversity_info['Native Hawaiian or \nOther Pacific Islander'] = pd.to_numeric(df_diversity_info['Native Hawaiian or \nOther Pacific Islander'], errors='coerce').fillna(0)
df_diversity_info['Total Enrollment'] = pd.to_numeric(df_diversity_info['Total Enrollment'], errors='coerce').fillna(1) # Avoid division by zero
df_diversity_info['URM%'] = (
    df_diversity_info['American Indian \nor Alaska Native'] +
    df_diversity_info['Black or \nAfrican American'] +
    df_diversity_info['Hispanic, Latino, or \nof Spanish Origin '] +
    df_diversity_info['Native Hawaiian or \nOther Pacific Islander']
) / df_diversity_info['Total Enrollment'] * 100

df_diversity_info.dropna(subset=['AAMC_Institution'], inplace=True)

### Next, let's parse class size (2025)
Source: https://www.aamc.org/data-reports/students-residents/data/facts-applicants-and-matriculants, Table A-1

In [7]:
import pandas as pd

ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"
file_path = os.path.join(ROOT_DIR, "data/raw_inputs/2025_FACTS_Table_A-1.xlsx")
df_class_size_info = pd.read_excel(file_path, sheet_name='FACTS Table A-1', header=4)
df_class_size_info = df_class_size_info[df_class_size_info['Matriculants'].notnull()]
df_class_size_info.drop(columns=['Applications'], inplace=True)
df_class_size_info.rename(columns={'Matriculants': 'Class Size', 'Applications by School': 'State', 'Unnamed: 1': 'name', 'Applications1':'Applications'}, inplace=True)
df_class_size_info = df_class_size_info[['name', 'Applications', 'Class Size']]
df_class_size_info.dropna(subset=['name'], inplace=True)
df_class_size_info['Matriculation Rate'] = df_class_size_info['Class Size'] / df_class_size_info['Applications'] * 100
df_class_size_info.sort_values(by='Matriculation Rate', ascending=True, inplace=True)

### Next, lets parse tuition and cost of attendance data
Source: https://students-residents.aamc.org/medical-school-admission-requirements/medical-school-admission-requirements-reports-applicants-and-advisors

In [ ]:
import camelot
import pandas as pd
import os

# Assuming ROOT_DIR is the same
ROOT_DIR = "/Users/jiturner/Repositories/medical-school-rankings/"

# 1. Update file path
file_path = os.path.join(ROOT_DIR, 'data/raw_inputs/MSAR015 - MSAR Tuition,Fees and Insurance Information.pdf')

# 2. Update page range
tables = camelot.read_pdf(file_path, pages='2-11', flavor='lattice')

processed_dfs = [] 

if tables.n > 0:
    print(f"Found {tables.n} tables across pages 2-11.")
    
    # 3. Loop logic updated
    for table in tables:
        my_df = table.df
        
        try:
            # --- START: New Header Logic ---
            
            # 1. Get both header rows
            super_headers = my_df.iloc[0] # Row 0: ['','', 'In-State', 'In-State', ...]
            main_headers = my_df.iloc[1]  # Row 1: ['','Medical School', 'Total Cost...', ...]
            
            # 2. Fill forward the super-headers so 'In-State' applies to all its columns
            super_headers.replace('', pd.NA, inplace=True)
            super_headers.ffill(inplace=True) 
            
            # 3. Create new column names
            new_cols = []
            for sup_head, main_head in zip(super_headers, main_headers):
                
                if pd.isna(sup_head):
                    # This is for 'State' (blank) and 'Medical School'
                    new_cols.append(main_head)
                else:
                    # This is for the In-State/Out-of-State columns
                    prefix = 'instate_' if 'In-State' in sup_head else 'outstate_'
                    # Clean up newlines in the header name itself
                    main_head_clean = main_head.replace('\n', ' ')
                    new_cols.append(f"{prefix}{main_head_clean}")
                    
            # 4. Manually set the first column (which is blank in both rows)
            if new_cols[0] == '':
                 new_cols[0] = 'State'
                 
            # 5. Set new columns and skip the 2 header rows
            my_df.columns = new_cols
            my_df = my_df[2:]
            
            # --- END: New Header Logic ---
            
            processed_dfs.append(my_df)
            
        except Exception as e:
            print(f"Warning: Could not process a table, possibly bad format? Error: {e}")
            print(f"Problematic table data:\n{table.df.head()}\n")
    
    # 4. Stack into a new DataFrame
    if processed_dfs:
        df_tuition_info = pd.concat(processed_dfs, ignore_index=True)
    else:
        df_tuition_info = pd.DataFrame() # Empty
        
else:
    print("No tables found in that page range.")
    df_tuition_info = pd.DataFrame()

df_tuition_info

# Convert all cost columns to numeric, removing $ and , first
cost_columns = [
    'instate_Total Cost of Attendance', 'instate_Health Insurance', 'instate_Tuition and Fees',
    'outstate_Total Cost of Attendance', 'outstate_Health Insurance', 'outstate_Tuition and Fees'
]
for col in cost_columns:
    df_tuition_info[col] = pd.to_numeric(df_tuition_info[col].str.replace('[\$,]', '', regex=True), errors='coerce')

# Manually update Albert Einstein College of Medicine tuition data
albert_einstein_tuition = 0
albert_einstein_insurance = 5465
albert_einstein_cost_of_attendance= 35465
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'instate_Total Cost of Attendance'] = albert_einstein_cost_of_attendance
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'instate_Health Insurance'] = albert_einstein_insurance
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'instate_Tuition and Fees'] = albert_einstein_tuition
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'outstate_Total Cost of Attendance'] = albert_einstein_cost_of_attendance
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'outstate_Health Insurance'] = albert_einstein_insurance
df_tuition_info.loc[df_tuition_info['Medical School']=='Albert Einstein College of Medicine', 'outstate_Tuition and Fees'] = albert_einstein_tuition

# Great! Now we want to consolidate the outstate/instate columns into single columns called 'Tuition and Fees', 'Health Insurance', and 'Total Cost of Attendance'
# That will be the MAXIMUM of the two columns for each row
df_tuition_info['Tuition and Fees'] = df_tuition_info[['instate_Tuition and Fees', 'outstate_Tuition and Fees']].max(axis=1)
df_tuition_info['Health Insurance'] = df_tuition_info[['instate_Health Insurance', 'outstate_Health Insurance']].max(axis=1)
df_tuition_info['Total Cost of Attendance'] = df_tuition_info[['instate_Total Cost of Attendance', 'outstate_Total Cost of Attendance']].max(axis=1)
# Now sort values by Total Cost of Attendance
df_tuition_info['Total Cost of Attendance'] = pd.to_numeric(df_tuition_info['Total Cost of Attendance'], errors='coerce')
df_tuition_info.sort_values(by='Total Cost of Attendance', ascending=True, inplace=True)
df_tuition_info.reset_index(drop=True, inplace=True)
df_tuition_info.rename(columns={'Medical School': 'name'}, inplace=True)
# Let's also get rid of \n characters in the 'name' column
df_tuition_info['name'] = df_tuition_info['name'].str.replace('\n', ' ', regex=False).str.strip()
df_tuition_info['name'] = df_tuition_info['name'].str.replace('  ', ' ', regex=False)
df_tuition_info['name'] = df_tuition_info['name'].str.strip()
# Finally, drop the now-unnecessary columns
df_tuition_info.drop(columns=['State', 'instate_Total Cost of Attendance', 'instate_Health Insurance', 'instate_Tuition and Fees',
                              'outstate_Total Cost of Attendance', 'outstate_Health Insurance', 'outstate_Tuition and Fees'], inplace=True)

<>:82: SyntaxWarning: invalid escape sequence '\$'
<>:82: SyntaxWarning: invalid escape sequence '\$'
/var/folders/76/y_3kwb_54mx0bh961wd_ycb00000gn/T/ipykernel_52969/1797820326.py:82: SyntaxWarning: invalid escape sequence '\$'
  df_tuition_info[col] = pd.to_numeric(df_tuition_info[col].str.replace('[\$,]', '', regex=True), errors='coerce')


Found 10 tables across pages 2-11.


array(['The University of Texas Health Science Center at San Antonio Joe R. and Teresa Lozano Long School of Medicine'],
      dtype=object)

### Figure out how to join all the data together

In [ ]:
import pandas as pd
from functools import reduce
import json
import numpy as np

# Let's list all the dfs we have so far, along with the key column to join on

dataframes = {
    'MCAT_GPA': [df_mcat_gpa, 'Medical School'],
    'Debt_Info': [df_debt_info, 'Medical School'],
    'NIH_Funding': [df_nih_funding, 'Name'],
    'Faculty_Info': [df_faculty_info, 'Medical School'],
    'Hospital_Rankings': [df_hospital_rankings_pivoted, 'Institution'],
    'Diversity': [df_diversity_info, 'AAMC_Institution'],
    'Class_Size': [df_class_size_info, 'name'],
    'Tuition': [df_tuition_info, 'name']
}


# Your data (assuming they are loaded)
df_mcat_gpa = dataframes['MCAT_GPA'][0]
df_debt_info = dataframes['Debt_Info'][0]
df_nih_funding = dataframes['NIH_Funding'][0]
df_faculty_info = dataframes['Faculty_Info'][0]
df_hospital_rankings = dataframes['Hospital_Rankings'][0]
df_diversity_info = dataframes['Diversity'][0]
df_class_size_info = dataframes['Class_Size'][0]
df_tuition_info = dataframes['Tuition'][0]


# 1. This is your "ground truth" list
canonical_schools = df_faculty_info['Medical School'].unique()

# 2. This is the "messy" list of *all* names from the other sources
messy_names = pd.concat([
    df_debt_info['Medical School'],
    df_nih_funding['Name'],
    df_mcat_gpa['Medical School'],
    df_hospital_rankings['Institution'],
    df_diversity_info['AAMC_Institution'],
    df_class_size_info['name'],
    df_tuition_info['name']
]).unique()

messy_names.tolist()

with open (os.path.join(ROOT_DIR, 'data/names_mapping.json'), 'r') as f:
    name_map = json.load(f)

for school in canonical_schools:
    name_map[school] = school

df_mcat_gpa['canonical_name'] = df_mcat_gpa['Medical School'].map(name_map)
df_mcat_gpa.dropna(subset=['canonical_name'], inplace=True)
df_debt_info['canonical_name'] = df_debt_info['Medical School'].map(name_map)
df_debt_info.dropna(subset=['canonical_name'], inplace=True)
df_nih_funding['canonical_name'] = df_nih_funding['Name'].map(name_map)
df_nih_funding.dropna(subset=['canonical_name'], inplace=True)
df_faculty_info['canonical_name'] = df_faculty_info['Medical School'].map(name_map)
df_faculty_info.dropna(subset=['canonical_name'], inplace=True)
df_hospital_rankings['canonical_name'] = df_hospital_rankings['Institution'].map(name_map)
df_hospital_rankings.dropna(subset=['canonical_name'], inplace=True)
df_diversity_info['canonical_name'] = df_diversity_info['AAMC_Institution'].map(name_map)
df_diversity_info.dropna(subset=['canonical_name'], inplace=True)
df_class_size_info['canonical_name'] = df_class_size_info['name'].map(name_map)
df_class_size_info.dropna(subset=['canonical_name'], inplace=True)
df_tuition_info['canonical_name'] = df_tuition_info['name'].map(name_map)
df_tuition_info.dropna(subset=['canonical_name'], inplace=True)

df_nih_funding # Let's see all rows with duplicate Canonical names
df_nih_funding_consolidated = df_nih_funding.copy()
df_nih_funding_consolidated[df_nih_funding_consolidated.duplicated(subset=['canonical_name'], keep=False)].sort_values(by='canonical_name')
df_nih_funding_consolidated = df_nih_funding_consolidated.groupby('canonical_name', as_index=False).agg({'School of Medicine Award': 'sum'})

hospital_grouped = df_hospital_rankings.groupby('canonical_name').min(numeric_only=True)
hospital_grouped.drop(columns=['#n_ranked_specialties', '#n_top10_specialties', '#n_top1_specialties'], inplace=True, errors='ignore')
hospital_grouped_with_cumulative_columns = hospital_grouped.copy()
hospital_grouped_with_cumulative_columns['#n_ranked_specialties'] = hospital_grouped.count(axis=1)
hospital_grouped_with_cumulative_columns['#n_top10_specialties'] = (hospital_grouped <= 10).sum(axis=1)
hospital_grouped_with_cumulative_columns['#n_top1_specialties'] = (hospital_grouped == 1).sum(axis=1)
hospital_grouped_with_cumulative_columns.loc['Weill Cornell Medicine'] = hospital_grouped_with_cumulative_columns.loc['Columbia University Vagelos College of Physicians and Surgeons']

all_dfs = [
    df_mcat_gpa, 
    df_debt_info, 
    df_nih_funding_consolidated,
    df_faculty_info, 
    hospital_grouped_with_cumulative_columns,
    df_diversity_info,
    df_class_size_info,
    df_tuition_info
]

# Drop any rows with 

final_df = reduce(lambda left, right: pd.merge(left, right, on='canonical_name', how='outer'), all_dfs)
final_df.sort_values(by='canonical_name', inplace=True)
final_df.rename(columns={'Average Graduate\nIndebtedness': 'Average Graduate Indebtedness', 'School of Medicine Award': 'NIH Research Funding'}, inplace=True)
final_df['NIH Research Funding per Faculty'] = final_df['NIH Research Funding'] / final_df['Total Faculty']
columns_we_care_about = ['AAMC_Institution', 'canonical_name', 'Degree Type', 'Average GPA', 'Average MCAT', 
                        'Average Graduate Indebtedness', 'Tuition and Fees', 'Total Cost of Attendance',
                        'NIH Research Funding', 'NIH Research Funding per Faculty', 'Total Faculty',
                        '#n_ranked_specialties', '#n_top10_specialties', '#n_top1_specialties',
                        'URM%',
                        'Applications',	'Class Size', 'Matriculation Rate',
                        'Cancer', 
                        'Cardiology, Heart & Vascular Surgery', 'Diabetes & Endocrinology',
                        'Cardiology, Heart & Vascular Surgery', 'Diabetes & Endocrinology',
                        'Ear, Nose & Throat', 'Gastroenterology & GI Surgery', 'Geriatrics',
                        'Neurology & Neurosurgery', 'Obstetrics & Gynecology', 'Ophthalmology',
                        'Orthopedics', 'Psychiatry', 'Pulmonology & Lung Surgery',
                        'Rehabilitation', 'Rheumatology', 'Urology']
final_df = final_df[columns_we_care_about]
final_df.drop_duplicates(inplace=True)

# Now let's do a complicated sorting approach-- sort by Average MCAT descending, then by average GPA descending.
final_df.sort_values(by=['Average MCAT', 'Average GPA'], ascending=[False, False], inplace=True)


final_df.to_csv(os.path.join(ROOT_DIR, 'data/final_medical_school_data.csv'), index=False)

final_df[['AAMC_Institution', 'Average Graduate Indebtedness', 'Total Cost of Attendance', 'Average MCAT', 'Average GPA']].head(20)

,AAMC_Institution,Average Graduate Indebtedness,Total Cost of Attendance,Average MCAT,Average GPA
59,NYU-Grossman,67572,33612.0,523.00,3.98
148,Vanderbilt,209382,114277.0,522.00,3.94
159,Yale,91965,111257.0,522.00,3.92
15,Columbia-Vagelos,114362,112753.0,522.00,3.90
68,Pennsylvania-Perelman,150137,112359.0,521.80,3.94
49,Mayo,144152,104034.0,521.00,3.94
38,Johns Hopkins,111516,102460.0,521.00,3.92
107,Chicago-Pritzker,127671,96345.0,521.00,3.91
33,Harvard,118957,107888.0,520.42,3.90
93,USF-Morsani,193357,88883.0,520.00,3.95
